In [67]:
import pandas as pd
import os
import xml.etree.ElementTree as ET
import datetime
import time

In [68]:
os.chdir(r"C:\Users\aman.khatri91425\Desktop\HomeCredit\Bigdata\CIBIL XML Masking")

In [69]:
xmlDf = pd.DataFrame()
for fl in os.listdir('Raw XML'): 
    cuid = str.split(fl,"_")[0]
    creationDt = str.split(fl,"_")[2].split(".")[0]
    whole_tree = ET.parse('Raw XML/' + fl)
    xml_string = ET.tostring(whole_tree.getroot())
    xmlDf = xmlDf.append({'cuid':cuid,'creation_date':creationDt,'xml_value':xml_string},  ignore_index=True)
    print(xml_string)

b'<CreditReport><Header><SegmentTag>TUEF</SegmentTag><Version>12</Version><ReferenceNumber>4004669814</ReferenceNumber><MemberCode>NB69281138&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;&#160;</MemberCode><SubjectReturnCode>1</SubjectReturnCode><EnquiryControlNumber>003428266285</EnquiryControlNumber><DateProcessed>08072020</DateProcessed><TimeProcessed>204540</TimeProcessed></Header><NameSegment><Length>03</Length><SegmentTag>N01</SegmentTag><ConsumerName1FieldLength>05</ConsumerName1FieldLength><ConsumerName1>FEROZ</ConsumerName1><ConsumerName3FieldLength>04</ConsumerName3FieldLength><ConsumerName3>KHAN</ConsumerName3><DateOfBirthFieldLength>08</DateOfBirthFieldLength><DateOfBirth>07041978</DateOfBirth><GenderFieldLength>01</GenderFieldLength><Gender>2</Gender></NameSegment><IDSegment><Length>03</Length><SegmentTag>I01</SegmentTag><IDType>03</IDType><IDNumberFieldLength>10</IDNumberFieldLength><IDNumber>IKR1171750</

In [70]:
xmlDf


,creation_date,cuid,xml_value
0,20200708,17340128,b'<CreditReport><Header><SegmentTag>TUEF</Segm...
1,20200708,17404218,b'<CreditReport><Header><SegmentTag>TUEF</Segm...
2,20200708,20828798,b'<CreditReport><Header><SegmentTag>TUEF</Segm...
3,20200708,20828845,b'<CreditReport><Header><SegmentTag>TUEF</Segm...
4,20200708,20828852,b'<CreditReport><Header><SegmentTag>TUEF</Segm...
5,20200708,20828873,b'<CreditReport><Header><SegmentTag>TUEF</Segm...
6,20200708,20828881,b'<CreditReport><Header><SegmentTag>TUEF</Segm...
7,20200708,20828887,b'<CreditReport><Header><SegmentTag>TUEF</Segm...
8,20200708,20828899,b'<CreditReport><Header><SegmentTag>TUEF</Segm...
9,20200708,5586129,b'<CreditReport><Header><SegmentTag>TUEF</Segm...


In [71]:
cibilSegmentsToRemove = ['NameSegment','IDSegment','TelephoneSegment','Address','EmailContactSegment', 'Header','End']
cibilSegmentsToKeep = ['ScoreSegment', 'EmploymentSegment','Account','Enquiry']
cibilAllSegments = cibilSegmentsToKeep + cibilSegmentsToRemove

In [58]:
cibilSubSegmentsToRemove = ['IDNumber','ConsumerName1','ConsumerName2','ConsumerName3',
                            'ConsumerName4','ConsumerName5','ConsumerName6','ConsumerName7',
                            'AddressLine1','AddressLine2','AddressLine3','AddressLine4',
                            'AddressLine5','AddressLine6','AddressLine7','MemberCode','EnquiryControlNumber',
                            'TelephoneNumber','EmailID']

In [72]:
cibilSegmentsToKeep

['ScoreSegment', 'EmploymentSegment', 'Account', 'Enquiry']

In [60]:
def exportXmlFileCibil(src, cuid, dtCreation, xml_message):
    tree = ET.ElementTree(ET.fromstring(xml_message))
    root = tree.getroot()
    newXml = ET.Element('CreditReport')

    for a in root:
        
        subElem = ET.Element(a.tag)
        arElement.append(a.tag)
        
        if a.tag in cibilSegmentsToKeep:
            newXml.insert(len(newXml),a)
        if a.tag not in cibilAllSegments:
            print(str(cuid) + ' : ' + a.tag + ' tag not recognized.')
              
    flName = src + '_' + str(cuid) + '_' + str(dtCreation) +'.xml'
    newTree = ET.ElementTree(newXml)
    tree.write(src + " XML\\" + flName)

In [73]:
def exportXmlFileCibilMasked(src, cuid, dtCreation, xml_message):
    tree = ET.ElementTree(ET.fromstring(xml_message))
    root = tree.getroot()
    newXml = ET.Element('CreditReport')

    for a in root:
        
        subElem = ET.Element(a.tag)
        arElement.append(a.tag)
        
        if a.tag in cibilSegmentsToRemove:
            for child in a:
                if child.tag in cibilSubSegmentsToRemove:
                    child.text = "******MASKED*******"
                        
        if a.tag not in cibilAllSegments:
            print(str(cuid) + ' : ' + a.tag + ' tag not recognized.')
        else :
            newXml.insert(len(newXml),a)
              
    flName = src + '_' + str(cuid) + '_' + str(dtCreation) +'.xml'
    newTree = ET.ElementTree(newXml)
    tree.write(src + " XML\\" + flName)

In [62]:
dtCibilSample = xmlDf.head(35)

In [63]:
arElement = []
dtStart = datetime.datetime.fromtimestamp(time.time())
res = dtCibilSample.apply(lambda x : exportXmlFileCibil('Unmasked', x['cuid'],x['creation_date'],x['xml_value']), axis = 1)
dtEnd = datetime.datetime.fromtimestamp(time.time())
print(dtStart.strftime('%Y-%m-%d %H:%M:%S'))
print(dtEnd.strftime('%Y-%m-%d %H:%M:%S'))
print(dtEnd - dtStart)

2020-07-15 16:42:23
2020-07-15 16:42:23
0:00:00.018948


In [74]:
arElement = []
dtStart = datetime.datetime.fromtimestamp(time.time())
res = dtCibilSample.apply(lambda x : exportXmlFileCibilMasked('MASKED', x['cuid'],x['creation_date'],x['xml_value']), axis = 1)
dtEnd = datetime.datetime.fromtimestamp(time.time())
print(dtStart.strftime('%Y-%m-%d %H:%M:%S'))
print(dtEnd.strftime('%Y-%m-%d %H:%M:%S'))
print(dtEnd - dtStart)

2020-07-15 16:59:34
2020-07-15 16:59:34
0:00:00.017953


In [51]:
distElement = list(dict.fromkeys(arElement))

In [52]:
len(distElement) == len(cibilAllSegments)

True